# Classification & Scoring

Classification

In [ ]:
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib
import statsmodels.api as sm
import xlsxwriter
import math
from statsmodels.regression.rolling import RollingOLS
import holoviews as hv

#data
data = pd.read_csv('aussie_equities_scoring_python_15072024.csv', index_col=0, encoding='latin-1')
data1 = data.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data2 = data1.copy()
data2.columns = [data1.columns, data.iloc[0]]
data1.columns = data.iloc[0]
data_cleaned = data1.iloc[1:]

#tracking error
excess_return = data_cleaned.iloc[:,5:].sub(data_cleaned.iloc[:,:1]['Market'], axis=0)
tracking_errors = excess_return.std() * math.sqrt(12)
tracking_errors.index = data2.iloc[:,5:].columns

#beta estimation
output = {}
output2 = {}
for i in data_cleaned.iloc[:,5:].columns:
    y = data_cleaned[i]
    x = data_cleaned.iloc[:,:5]
    if y.count() < 24:
            output[i] = np.nan
            output2[i] = np.nan
    else: 
        model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
        output[i] = model.params
        output2[i] = model.tvalues

coefficients = pd.DataFrame(output)
tstats = pd.DataFrame(output2)
coefficients.columns = data2.iloc[:,5:].columns
tstats.columns = data2.iloc[:,5:].columns

#classification
classifications = []
for i in tstats.columns:
    ranks = tstats[i].sort_values(axis=0, ascending = False)
    ranks_cleaned = ranks.drop(['Market','const'],axis=0)

    if tracking_errors.loc[i] < 0.01:
        classifications.append('Beta')
        
    elif ranks_cleaned.iloc[0] > 2.581:
        if not (ranks_cleaned.index[-1] == 'Value' and ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
            classifications.append(ranks_cleaned.index[0])
        else:
            classifications.append('Growth')

    elif (ranks_cleaned.loc['Value'] < -2.581 and (abs(ranks_cleaned.loc['Value']) > ranks_cleaned.iloc[0])):
        classifications.append('Growth')

    elif tracking_errors.loc[i] < 0.02:
        classifications.append('Beta')

    else:
        classifications.append('Alpha')
    

final_df = tstats.T
final_df['Classification'] = classifications
final_df['Tracking Error'] = tracking_errors

Find:

In [1]:
# final_df.xs('DNR Capital Aus Eq High Conviction R',axis=0,level=0)

Rolling visualization:

In [ ]:
exog = sm.add_constant(data_cleaned.iloc[:,:5], prepend=False)
mod = RollingOLS(data_cleaned['PIM0028AU'], exog, window=36, missing='drop').fit()
mod.params.index = pd.to_datetime(mod.params.index)
plot = mod.params.dropna().iloc[10:,1:-1].hvplot(rot=90, title='DNR', width=800, height=500)
hline = hv.HLine(0).opts(color='gray',line_width = 0.75)
combo = plot * hline
combo

Actual Scoring:

In [ ]:
df_import = pd.read_csv('aussie_equities_scoring_python_15072024.csv', index_col=0, encoding='latin-1')
data1 = df_import.apply(lambda x: pd.to_numeric(x, errors='coerce'))
data2 = data1.copy()
data2.columns = [data1.columns, df_import.iloc[0]]
data_cleaned = data2.iloc[1:]

df_export = final_df #pd.read_csv('new_scoring.csv')

#create a classification df
classification = df_export[['Unnamed: 0', 'apir_ticker', 'Classification']]
classification.index = classification['apir_ticker']

#beta estimation
output = {}
output2 = {}

data_drop_level_0 = data_cleaned.droplevel(axis=1,level=0)

for i in data_drop_level_0.iloc[:,5:].columns:
    y = data_drop_level_0.loc[:,i].iloc[-120:]
    
    if classification.loc[i]['Classification'] == 'Beta' or classification.loc[i]['Classification'] == 'Alpha':
        x = data_drop_level_0.loc[:,'Market'].iloc[-120:]

    else: 
        classification_value = classification.loc[i]['Classification']
        if classification_value == 'Growth':
            columns_to_select = ['Market', 'Value']
            x = data_drop_level_0.loc[:, columns_to_select].iloc[-120:]
            x['Value'] = -x['Value']
            x = x.rename(columns={'Value':'Growth'})

        else:
            columns_to_select = ['Market', classification_value]
            x = data_drop_level_0.loc[:, columns_to_select].iloc[-120:]

    if y.count() < 24: 
            output[i] = np.nan
            output2[i] = np.nan
        
    else: 
        model = sm.OLS(y, sm.add_constant(x), missing='drop').fit()
        output[i] = model.params
        output2[i] = model.rsquared

coefficients = pd.DataFrame(output)
coefficients.columns = data2.iloc[:,5:].columns
coefficients = coefficients.T
coefs = coefficients.reset_index()
coefs.index = coefs['apir_ticker']
coefs['classification'] = classification['Classification']